In [7]:
import pandas as pd
pd.Series
import math
import csv
from datetime import datetime
from datetime import date
import numpy as np
import scipy as sc
import statsmodels
import sklearn
from sklearn import preprocessing
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 30, 10
rcParams.update({'font.size': 22})
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from matplotlib import pyplot
import statsmodels.api as sm

import holidays

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

### Load file, print info and select columns

In [2]:
#function to load files
def load_file(filepath):
    df = pd.read_csv(filepath, sep='\t', index_col=0, parse_dates=True)
    df = df.sort_index()
    #we can check that this 2 columns are equal, so we can drop one
    #any(df['SALE_AMOUNT_BEFORE_CANCELLATIONS'] != df['SALE_AMOUNT_AFTER_CANCELLATIONS'])
    df = df.drop(['SALE_AMOUNT_AFTER_CANCELLATIONS', 'WEEKDAY'], axis=1)
    return df.astype('float32')

def load_file_exp(filepath):
    df = pd.read_csv(filepath, sep=',', index_col=0, parse_dates=True)
    df = df.sort_index()
    return df.astype('float32')

def add_timeindex_features(df):
    data_df = df.copy()
    data_df['month'] = data_df.index.month
    data_df['year'] = data_df.index.year
    data_df['quater'] = data_df.index.quarter
    data_df['week_day'] = data_df.index.dayofweek
    data_df['week_of_year'] = data_df.index.weekofyear
    data_df['day_of_year'] = data_df.index.dayofyear
    return data_df

#function to create a new df with selected columns
def create_small_df(df, columns):
    small_df = df.copy()
    small_df = small_df[columns]
    return small_df

#function to print inf about Data
def print_info_df(df, print_columns = False):
    #Count period
    d1 = df.index[0]
    d2 = df.index[-1]
    delta = d2 - d1
    print('Number of days is ' + str(delta.days) + ' from ' + str(d1) + ' to '+ str(d2))
    print('The shape of the data: %d*%d' %(df.shape[0],df.shape[1]))
    print('Check for Nan values: %s'%(df.isnull().values.any()))
    if (print_columns == True):
        print(list(df.columns))
    else:
        print('Number of columns: %d'%(df.shape[1]))

In [ ]:
df = load_file('...')
print_info_df(df, False)

### UK holidays

In [4]:
uk_holidays = holidays.UK()

In [5]:
 for date, name in sorted(holidays.UK(years=2018).items()):
     print(date, name)

2018-01-01 New Year's Day
2018-01-02 New Year Holiday [Scotland]
2018-03-17 St. Patrick's Day [Northern Ireland]
2018-03-19 St. Patrick's Day [Northern Ireland] (Observed)
2018-03-30 Good Friday
2018-04-02 Easter Monday [England, Wales, Northern Ireland]
2018-05-07 May Day
2018-05-28 Spring Bank Holiday
2018-07-12 Battle of the Boyne [Northern Ireland]
2018-08-06 Summer Bank Holiday [Scotland]
2018-08-27 Late Summer Bank Holiday [England, Wales, Northern Ireland]
2018-11-30 St. Andrew's Day [Scotland]
2018-12-25 Christmas Day
2018-12-26 Boxing Day


### Special E-Commerce days 

In [8]:
#Define new class
class ECommerceDays(holidays.HolidayBase):
    def _populate(self, year):
        
        self[date(year, 2, 13)] = "preValentine Day" 
        self[date(year, 2, 14)] = "Valentine Day"       
        self[date(year, 12, 15)] = "Free Shipping day"
        self[date(year, 12, 26)] = "Boxing Day"
        self[date(year, 1, 15)] = "Blue Monday"
        
ecomdays = ECommerceDays()
ecomdays._populate(2014)
ecomdays._populate(2015)
ecomdays._populate(2016)
ecomdays._populate(2017)
ecomdays._populate(2018)
ecomdays._populate(2019)

to_add = ({date(2014, 11, 28) : "Black Friday 2014"},{date(2015, 11, 27) : "Black Friday 2015"},
          {date(2016, 11, 25) : "Black Friday 2016"},{date(2017, 11, 24) : "Black Friday 2017"},
          {date(2018, 11, 23) : "Black Friday 2018"},{date(2019, 11, 29) : "Black Friday 2019"},
          {date(2014, 12, 6) : "Small Business day 2014"},{date(2015, 12, 5) : "Small Business day 2015"},
          {date(2016, 12, 3) : "Small Business day 2016"},{date(2017, 12, 2) : "Small Business day 2017"},
          {date(2018, 12, 1) : "Small Business day 2018"},{date(2019, 12, 7) : "Small Business day 2019"})

for i in to_add:
    ecomdays.append(i)

In [9]:
# for date in (ecomdays.items()):
#     print(date)

### Christmas  days

In [10]:
class ChristmasDays(holidays.HolidayBase):
    def _populate(self, year):
        
        self[date(year, 12, 1)] = "Presents for Christmas"
        self[date(year, 12, 2)] = "Presents for Christmas"
        self[date(year, 12, 3)] = "Presents for Christmas"
        self[date(year, 12, 4)] = "Presents for Christmas"
        self[date(year, 12, 5)] = "Presents for Christmas"
        self[date(year, 12, 6)] = "Presents for Christmas"
        self[date(year, 12, 7)] = "Presents for Christmas"
        self[date(year, 12, 8)] = "Presents for Christmas"
        self[date(year, 12, 9)] = "Presents for Christmas"
        self[date(year, 12, 10)] = "Presents for Christmas"
        self[date(year, 12, 11)] = "Presents for Christmas"
        self[date(year, 12, 12)] = "Presents for Christmas"
        self[date(year, 12, 13)] = "Presents for Christmas"
        self[date(year, 12, 14)] = "Presents for Christmas"
        self[date(year, 12, 15)] = "Presents for Christmas"
        self[date(year, 12, 16)] = "Presents for Christmas"
        self[date(year, 12, 17)] = "Presents for Christmas"
        self[date(year, 12, 18)] = "Presents for Christmas"
        self[date(year, 12, 19)] = "Presents for Christmas"
        self[date(year, 12, 20)] = "Presents for Christmas"
        self[date(year, 12, 21)] = "Presents for Christmas"
        self[date(year, 12, 22)] = "Presents for Christmas"
        self[date(year, 12, 23)] = "Presents for Christmas"
        
christmas_days = ChristmasDays()
christmas_days._populate(2014)
christmas_days._populate(2015)
christmas_days._populate(2016)
christmas_days._populate(2017)
christmas_days._populate(2018)
christmas_days._populate(2019)

In [11]:
# for date in (christmas_days.items()):
#      print(date)

### Now we have to add it 

In [12]:
def add_new_features(df):
    ex_df = df.copy()
    ex_df['uk_holidays'] = 0
    ex_df['ecomdays'] = 0
    ex_df['christmas_days'] = 0
    
    #add uk holidays
    for i in ex_df.index:
        date=i.strftime('%-m/%d/%y')
        if date in uk_holidays:
            ex_df.at[i, 'uk_holidays'] = 1
        else:
            ex_df.at[i, 'uk_holidays'] = 0
        
    #add special e-commerce days    
    for i in ex_df.index:
        date=i.strftime('%-m/%d/%y')
        if date in ecomdays:
            ex_df.at[i, 'ecomdays'] = 1
        else:
            ex_df.at[i, 'ecomdays'] = 0
        
    #add christmas days
    for i in ex_df.index:
        date=i.strftime('%-m/%d/%y')
        if date in christmas_days:
            ex_df.at[i, 'christmas_days'] = 1
        else:
            ex_df.at[i, 'christmas_days'] = 0
     
    #add month, year, quater, dayofweek, weekofyear, day_of_year
    ex_df = add_timeindex_features(ex_df)
    
    return ex_df

In [ ]:
new_df = add_new_features(df)
print_info_df(new_df, True)

In [119]:
export_csv = new_df.to_csv (r'export2_dataframe.csv', index = 'DATE', header=True)

In [122]:
our_df = load_file_exp('export2_dataframe.csv', )

In [128]:
df our_df = load_file_exp('export2_dataframe.csv', )

(1796, 14)

In [129]:
export_csv = df.to_csv (r'selected_dataframe.csv', index = 'DATE', header=True)

In [ ]:
our_df = load_file_exp('selected_dataframe.csv', )